# Yield calculator

This notebook is used to estimate the CCSNe yields consistent with the present-day APOGEE trend in \[C/Mg\]-\[Mg/H\].

uses the results from the MCMC fits to AGB integrated yields with 

In [1]:
import toml
import surp
import vice

import vice.yields.ccsne.settings as y_cc
import vice.yields.agb.settings as Y_agb
import vice.yields.sneia.settings as y_ia

/astro/dboyea/.local/lib/python3.11/site-packages/vice-1.4.0.dev0-py3.11-linux-x86_64.egg/vice/__init__.py:141: UserWarning: Using un-released version of VICE
  warnings.warn("Using un-released version of VICE", UserWarning)


In [2]:
surp.set_yields(verbose=False)

# yield params calculator

Given the desired f_agb or alpha, what are the required y0cc and alpha_agb values

## inputs from mcmc fits

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("yield_fits.tsv", sep=r"\s+", comment="#")
df.set_index("model", inplace=True)
df = df * 1e-4
df

,y0,zeta0,zeta0_err,zeta1,zeta1_err
model,,,,,
fruity,3.110000e-04,0.00037,0.00003,-0.00035,0.00003
aton,-9.000000e-07,0.00010,0.00019,-0.00100,0.00030
monash,2.960000e-04,0.00023,0.00005,-0.00110,0.00010
nugrid,9.250000e-04,0.00081,0.00019,-0.00046,0.00017
fruity_mf0.5,2.830000e-04,0.00033,0.00002,-0.00018,0.00002
fruity_mf0.7,3.600000e-04,0.00042,0.00003,-0.00031,0.00003
fruity_mf1.5,3.110000e-04,0.00034,0.00002,-0.00030,0.00002


In [5]:
# tuned to fruity
y_mg = y_cc["mg"]
Y_C_0 = 2.79e-3 # pm 0.16
ZETA_C_0 = 1.43e-3 # pm 0.32 (for fruity)
A_C_0 = 2.58 # pm 0.5

In [6]:
Y_C_AGB = {
    "cristallo11": df.loc["fruity", "y0"],
    "ventura13":  df.loc["aton", "y0"],
    "karakas16":  df.loc["monash", "y0"],
    "pignatari16":  df.loc["nugrid", "y0"],
    "cristallo11_mf0.5": df.loc["fruity_mf0.5", "y0"],
    "cristallo11_mf0.7": df.loc["fruity_mf0.7", "y0"],
    "cristallo11_mf1.5": df.loc["fruity_mf1.5", "y0"],

}

In [7]:
for label, yagb in Y_C_AGB.items():
    print(label, "\t", yagb / Y_C_0)

cristallo11 	 0.11146953405017922
ventura13 	 -0.0003225806451612903
karakas16 	 0.1060931899641577
pignatari16 	 0.33154121863799285
cristallo11_mf0.5 	 0.1014336917562724
cristallo11_mf0.7 	 0.12903225806451615
cristallo11_mf1.5 	 0.11146953405017922


In [8]:
ZETA_C_AGB = {
    "cristallo11":  df.loc["fruity", "zeta1"],
    "ventura13":df.loc["aton", "zeta1"],
    "karakas16": df.loc["monash", "zeta1"],
    "pignatari16": df.loc["nugrid", "zeta1"],
    "cristallo11_mf0.5": df.loc["fruity_mf0.5", "zeta1"],
    "cristallo11_mf0.7": df.loc["fruity_mf0.7", "zeta1"],
    "cristallo11_mf1.5": df.loc["fruity_mf1.5", "zeta1"],
}

## Utils

In [9]:
def f_agb_to_alpha(y0_agb, f_agb):
    y_agb = Y_C_0 * f_agb
    return y_agb / y0_agb

In [10]:
def f_agb_to_settings(study, y0=None, f_agb=0.2):
    if y0 is None:
        y0 = Y_C_AGB[study]
        
    alpha = f_agb_to_alpha(y0, f_agb)
    y0_cc = (1-f_agb) * Y_C_0
    settings = dict(alpha_c_agb = float(alpha), y0_c_cc = float(y0_cc))
    
    print(toml.dumps(settings))

In [11]:
def f_agb_zeta_to_settings(study, *, y0_agb=None, zeta_agb=None, f_agb=0.2):
    if y0_agb is None:
        y0_agb = Y_C_AGB[study]
    if zeta_agb is None:
        zeta_agb = ZETA_C_AGB[study]
        
    alpha = f_agb_to_alpha(y0_agb, f_agb)
    y0_cc = (1-f_agb) * Y_C_0
    
    zeta_cc = ZETA_C_0 - zeta_agb * alpha
    
    settings = dict(alpha_c_agb = float(alpha), y0_c_cc = float(y0_cc), zeta_c_cc=float(zeta_cc))
    
    print(toml.dumps(settings))

In [12]:
def alpha_to_settings(study, alpha):
    f_agb =  alpha * Y_C_AGB[study] / Y_C_0
    y0_cc = (1-f_agb) * Y_C_0
    settings = dict(alpha_c_agb = alpha, y0_c_cc = y0_cc)
    
    print(toml.dumps(settings))

In [13]:
def enhance_fe_ia(factor):
    """if factor != 1, then enhances sne ia fe yield by factor but maintains same total fe yield, applied to params"""
    if factor == 1:
        return
    y_fe = y_ia["fe"] + y_cc["fe"]
    y_fe_ia = y_ia["fe"] * factor
    y_fe_cc = y_fe - y_fe_ia
    
    print(f"y_fe_ia = {y_fe_ia:0.8f}")
    print(f"y_fe_cc = {y_fe_cc:0.8f}")
    return y_fe_ia, y_fe_cc


In [14]:
def m_factor_to_settings(Y_c_agb="cristallo11"):
    surp.set_yields(y_c_cc=0, Y_c_agb=Y_c_agb, kwargs_c_agb = dict(mass_factor=m), verbose=False)
    y_agb = surp.yields.calc_y(surp.Z_SUN)
    
    print(y_cc_0 - surp.yields.calc_y(surp.Z_SUN) + y_agb_0)

## Fruity

In [15]:
alpha_to_settings("cristallo11", alpha=1.449)

alpha_c_agb = 1.449
y0_c_cc = "np.float64(0.002339361)"



In [16]:
enhance_fe_ia(1.2)

y_fe_ia = 0.00092400
y_fe_cc = 0.00031900


(0.0009239999999999999, 0.00031900000000000006)

In [17]:
for f in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.8, 1]:
    print("f_agb = ", f)
    f_agb_zeta_to_settings("cristallo11", f_agb=f)
    print()

f_agb =  0
alpha_c_agb = 0.0
y0_c_cc = 0.00279
zeta_c_cc = 0.00143


f_agb =  0.1
alpha_c_agb = 0.8971061093247588
y0_c_cc = 0.002511
zeta_c_cc = 0.0017439871382636655


f_agb =  0.2
alpha_c_agb = 1.7942122186495175
y0_c_cc = 0.002232
zeta_c_cc = 0.002057974276527331


f_agb =  0.3
alpha_c_agb = 2.691318327974276
y0_c_cc = 0.0019529999999999999
zeta_c_cc = 0.002371961414790997


f_agb =  0.4
alpha_c_agb = 3.588424437299035
y0_c_cc = 0.001674
zeta_c_cc = 0.0026859485530546625


f_agb =  0.5
alpha_c_agb = 4.485530546623794
y0_c_cc = 0.001395
zeta_c_cc = 0.0029999356913183277


f_agb =  0.8
alpha_c_agb = 7.17684887459807
y0_c_cc = 0.0005579999999999999
zeta_c_cc = 0.003941897106109324


f_agb =  1
alpha_c_agb = 8.971061093247588
y0_c_cc = 0.0
zeta_c_cc = 0.0045698713826366555




In [19]:
for mf in [0.5, 0.7, 1.5]:
    print("mf = ", mf)
    f_agb_zeta_to_settings(f"cristallo11_mf{mf}", f_agb=0.317)
    print()

mf =  0.5
alpha_c_agb = 3.1251943462897525
y0_c_cc = 0.00190557
zeta_c_cc = 0.0019925349823321555


mf =  0.7
alpha_c_agb = 2.4567499999999995
y0_c_cc = 0.00190557
zeta_c_cc = 0.0021915924999999998


mf =  1.5
alpha_c_agb = 2.8438263665594854
y0_c_cc = 0.00190557
zeta_c_cc = 0.002283147909967846




## ATON

In [18]:
f_agb_zeta_to_settings("ventura13", f_agb=0.2)


alpha_c_agb = "np.float64(5.58)"
y0_c_cc = 0.002232
zeta_c_cc = "np.float64(0.00701)"



In [19]:
alpha_to_settings("ventura13", alpha=2)
print()
alpha_to_settings("ventura13", alpha=3)

alpha_c_agb = 2
y0_c_cc = "np.float64(0.00259)"


alpha_c_agb = 3
y0_c_cc = "np.float64(0.00249)"



## Monash

In [20]:
f_agb_zeta_to_settings("karakas16", f_agb=0.2)


alpha_c_agb = "np.float64(2.4260869565217393)"
y0_c_cc = 0.002232
zeta_c_cc = "np.float64(0.004098695652173913)"



In [21]:
for f in [0, 0.1, 0.3, 0.4, 0.5, 1]:
    print(f)
    f_agb_zeta_to_settings("karakas16", f_agb=f)

0
alpha_c_agb = "np.float64(0.0)"
y0_c_cc = 0.00279
zeta_c_cc = "np.float64(0.00143)"

0.1
alpha_c_agb = "np.float64(1.2130434782608697)"
y0_c_cc = 0.002511
zeta_c_cc = "np.float64(0.002764347826086957)"

0.3
alpha_c_agb = "np.float64(3.639130434782609)"
y0_c_cc = 0.0019529999999999999
zeta_c_cc = "np.float64(0.0054330434782608695)"

0.4
alpha_c_agb = "np.float64(4.852173913043479)"
y0_c_cc = 0.001674
zeta_c_cc = "np.float64(0.006767391304347827)"

0.5
alpha_c_agb = "np.float64(6.065217391304349)"
y0_c_cc = 0.001395
zeta_c_cc = "np.float64(0.008101739130434785)"

1
alpha_c_agb = "np.float64(12.130434782608697)"
y0_c_cc = 0.0
zeta_c_cc = "np.float64(0.014773478260869569)"



## NuGrid

In [22]:
f_agb_zeta_to_settings("pignatari16", f_agb=0.2)

alpha_c_agb = "np.float64(0.6888888888888889)"
y0_c_cc = 0.002232
zeta_c_cc = "np.float64(0.001746888888888889)"



In [23]:
for f in [0, 0.1, 0.3, 0.5, 1]:
    f_agb_zeta_to_settings("pignatari16", f_agb=f)

alpha_c_agb = "np.float64(0.0)"
y0_c_cc = 0.00279
zeta_c_cc = "np.float64(0.00143)"

alpha_c_agb = "np.float64(0.34444444444444444)"
y0_c_cc = 0.002511
zeta_c_cc = "np.float64(0.0015884444444444444)"

alpha_c_agb = "np.float64(1.0333333333333334)"
y0_c_cc = 0.0019529999999999999
zeta_c_cc = "np.float64(0.0019053333333333335)"

alpha_c_agb = "np.float64(1.7222222222222223)"
y0_c_cc = 0.001395
zeta_c_cc = "np.float64(0.0022222222222222222)"

alpha_c_agb = "np.float64(3.4444444444444446)"
y0_c_cc = 0.0
zeta_c_cc = "np.float64(0.0030144444444444446)"



## Analytic

In [ ]:
y0_agb_a = Y_C_AGB["cristallo11"]
y0_agb_a

In [ ]:
zeta_agb_a = ZETA_C_AGB["cristallo11"]
zeta_agb_a

In [ ]:
def get_alpha(f_agb=0.2, y0_agb=y0_agb_a):
    return Y_C_0 / y0_agb * f_agb

In [ ]:
def get_zeta_cc(alpha, zeta=zeta_agb_a):
    return ZETA_C_0 - alpha*zeta

In [ ]:
def get_ana_settings(f, zeta=zeta_agb_a):
    alpha = get_alpha(f)
    print("alpha_c_agb = ", alpha)
    print("y0_c_cc = ", Y_C_0 - alpha * y0_agb_a)
    print("zeta_c_cc = ", get_zeta_cc(alpha, zeta))

In [ ]:
get_ana_settings(0.2)

In [ ]:
get_ana_settings(0.2, 0)

In [ ]:
get_ana_settings(0.2, -0.0007)

In [ ]:
for f in [0, 0.1, 0.3, 0.5, 1]:
    print(f)
    get_ana_settings(f)
    print()

In [ ]:
y = surp.yield_models.C_AGB_Model()

In [ ]:
print(y)

In [ ]:
print(2*y)

In [ ]:
print(0.15 * y)